In [1]:
#Load important Libraries
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import shutil # to save it locally

In [2]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5','referer':'https://www.google.com/'})

In [3]:
data = pd.read_csv("products_data.csv")
data.head()

,name,link
0,"Kindle (10th Gen), 6"" Display with Built-in Li...",https://amazon.in/Staging-Product-Not-Retail-S...
1,Kindle Paperwhite (10th gen) - with Built-in L...,https://amazon.in/All-New-Kindle-Paperwhite-10...
2,"Kindle (10th Gen), 6"" Display with Built-in Li...",https://amazon.in/Staging-Product-Not-Retail-S...
3,Kindle Paperwhite (10th gen) -with Built-in Li...,https://amazon.in/All-New-Kindle-Paperwhite-10...
4,All-New Kindle Oasis (10th Gen) - Now with adj...,https://amazon.in/All-new-Kindle-Oasis/dp/B07L...


In [4]:
import re
# remove special characters
def remove_special_from_price(text):
    return re.sub('[^a-zA-Z0-9.]', '', text)
def remove_backsklashn(text):
    return re.sub('[\n]', '', text)
def remove_spaces(text):
    return text.strip()

In [5]:
products_info = pd.DataFrame(columns=['name','category','price','seller','return_exchange','warranty','no_of_images'])
products_info

,name,category,price,seller,return_exchange,warranty,no_of_images


In [6]:
dirCount = 0
dirSav = False
savPreset = False
for i,j in data.iterrows():
    count = 0
    link = j['link']
    page = requests.get(link, headers=HEADERS)
    if str(page)=="<Response [200]>":
        soup = BeautifulSoup(page.content)
        productStatus = soup.find('div',id = 'outOfStock')
        # Check Out of Stock -> 2nd Type
        if productStatus==None:
            productCategory = soup.find('span',class_='subnav-home')
            # Check Amazon Devices Products
            if productCategory==None:
                productName = soup.find('span',id='productTitle')
                productName = productName.text.replace('\n','')
                productCategory = soup.find('span',class_='nav-a-content')
                productCategory = remove_spaces(remove_backsklashn(productCategory.text))
                productPrice = soup.find('div',id='soldByThirdParty')
                if productPrice == None:
                    productPrice = 'N.A.'
                else:
                    productPrice = remove_special_from_price(productPrice.text)
                sellerName = soup.find('a',id='sellerProfileTriggerId')
                if sellerName==None:
                    sellerName='N.A.'
                else:
                    sellerName = sellerName.text
                tandc = soup.findAll('div',class_='icon-container')
                if len(tandc)==0:
                    returnpolicy='N.A.'
                    warranty='N.A.'
                else:
                    for icon in tandc:
                        if "replacement" not in icon.text.lower() and "refund" not in icon.text.lower() and "warranty" not in icon.text.lower():
                            returnpolicy='N.A.'
                            warranty='N.A.'
                        else:
                            if "replacement" in icon.text.lower() or "refund" in icon.text.lower():
                                returnpolicy = remove_spaces(remove_backsklashn(icon.text))
                            if "warranty" in icon.text.lower() and "days" not in icon.text.lower():
                                warranty = soup.find('div',class_='a-section a-spacing-none icon-farm-bottom-sheet-content')
                                warranty = remove_spaces(remove_backsklashn(warranty.text))
                            else:
                                warranty = remove_spaces(remove_backsklashn(icon.text))
                #Finding First Image URL using BeautifulSoup
                firstImage = soup.find('img',class_='frontImage')
                firstImage = firstImage['src'].split('.')
                #Finding All Span Elements containing image URLs using BeautifulSoup
                images = soup.find_all('div',class_='imageThumb')
                #List for Saving Image Src Links
                imageLinks = [firstImage[0]+'.'+firstImage[1]+'.'+firstImage[2]+'.'+firstImage[4]]
                for e in range(len(images)-1):
                    f = images[e+1].find('img')
                    #Removing image Elements that don't include image
                    if f is not None:
                        #Splitting Image Sources to remove all images except jpg extension
                        b = f['src'].split('.')
                        #Removing image Elements with gif extension
                        if(b[-1]!='gif'):
                            #Appending final image URL to a list
                            imageLinks.append(b[0]+'.'+b[1]+'.'+b[2]+'.'+b[4])
            # If Not Books Products
            else:
                productName = soup.find('span',id='productTitle')
                productName = productName.text.replace('\n','')
                productCategory = remove_spaces(remove_backsklashn(productCategory.text))
                productPrice = soup.find('span',class_='priceBlockBuyingPriceString')
                productPrice = remove_special_from_price(productPrice.text)
                sellerName = soup.find('a',id='sellerProfileTriggerId')
                sellerName = sellerName.text
                tandc = soup.findAll('div',class_='icon-container')
                returnpolicy = remove_spaces(remove_backsklashn(tandc[0].text))
                warranty = remove_spaces(remove_backsklashn(tandc[2].text))
                #Finding First Image URL using BeautifulSoup
                firstImage = soup.find('img',id='landingImage')
                firstImage = firstImage['data-a-dynamic-image'].split('"')
                firstImage = firstImage[1]
                firstImage = firstImage.split('.')
                #Finding All Span Elements containing image URLs using BeautifulSoup
                images = soup.find_all('span',class_='a-button a-button-thumbnail a-button-toggle')
                #List for Saving Image Src Links
                imageLinks = [firstImage[0]+'.'+firstImage[1]+'.'+firstImage[2]+'.'+firstImage[4]]
                for e in images:
                    f = e.find('img')
                    #Removing image Elements that don't include image
                    if f is not None:
                        #Splitting Image Sources to remove all images except jpg extension
                        b = f['src'].split('.')
                        #Removing image Elements with gif extension
                        if(b[-1]!='gif'):
                            #Appending final image URL to a list
                            imageLinks.append(b[0]+'.'+b[1]+'.'+b[2]+'.'+b[4])
            # Check if Directory is already there after scraping all image links
            if os.path.isdir(productName)==False:
                os.mkdir(productName)
                for element in imageLinks:
                    filename=productName+'/'+str(count+1)+'.jpg'
                    r = requests.get(element, stream = True, headers=HEADERS)
                    if r.status_code == 200:
                        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                        r.raw.decode_content = True
                        # Open a local file with wb ( write binary ) permission.
                        with open(filename,'wb') as f:
                            shutil.copyfileobj(r.raw, f)
                        count=count+1
                print(str(count)+' Images Downloaded Successfully')
            # If Directory for Images are already there
            else:
                # Check if Directory already message has been occured once
                if dirCount==1:
                    print(productName,'Directory already exists! Do you want to overwrite?(y/n)')
                    answer = input()
                    preset = input('Save this preset?(y/n)')
                    if answer=='y' or answer=="yes":
                        print(productName,'Directory Overwriting.')
                        for f in os.listdir(productName):
                            os.remove(os.path.join(productName, f))
                        for element in imageLinks:
                            filename=productName+'/'+str(count+1)+'.jpg'
                            r = requests.get(element, stream = True, headers=HEADERS)
                            if r.status_code == 200:
                                # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                                r.raw.decode_content = True
                                # Open a local file with wb ( write binary ) permission.
                                with open(filename,'wb') as f:
                                    shutil.copyfileobj(r.raw, f)
                                count=count+1
                        print(str(count)+' Images Downloaded Successfully')
                        if preset=='y' or preset=='yes':
                            dirSav = True
                            dirCount+=1
                        else:
                            savPreset = True
                            dirCount-=1
                    else:
                        print(productName,'Directory Skipping.')
                        if preset=='y' or preset=='yes':
                            dirSav = False
                            dirCount+=1
                        else:
                            savPreset = True
                            dirCount-=1
                # Check if Directory already message has been occured more than (>) once
                elif dirCount>1:
                    dirCount=dirCount+1
                    if dirSav==True:
                        print(productName,'Directory already exists! Overwriting.')
                        for f in os.listdir(productName):
                            os.remove(os.path.join(productName, f))
                        for element in imageLinks:
                            filename=productName+'/'+str(count+1)+'.jpg'
                            r = requests.get(element, stream = True, headers=HEADERS)
                            if r.status_code == 200:
                                # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                                r.raw.decode_content = True
                                # Open a local file with wb ( write binary ) permission.
                                with open(filename,'wb') as f:
                                    shutil.copyfileobj(r.raw, f)
                                count=count+1
                        print(str(count)+' Images Downloaded Successfully')
                    else:
                        print(productName,'Directory already exists! Skipping.')
                # First Iteration for Directory Already Exists
                else:
                    if savPreset==True:
                        print(productName,'Directory already exists! Do you want to overwrite?(y/n)')
                        answer = input()
                        # If saving directory overwrite options is set to yes.
                        if answer=='y' or answer=="yes":
                            for f in os.listdir(productName):
                                os.remove(os.path.join(productName, f))
                            for element in imageLinks:
                                filename=productName+'/'+str(count+1)+'.jpg'
                                r = requests.get(element, stream = True, headers=HEADERS)
                                if r.status_code == 200:
                                    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                                    r.raw.decode_content = True
                                    # Open a local file with wb ( write binary ) permission.
                                    with open(filename,'wb') as f:
                                        shutil.copyfileobj(r.raw, f)
                                    count=count+1
                            print(str(count)+' Images Downloaded Successfully')
                        # If saving directory overwrite options is set to no.
                        else:
                            print(productName,'Directory Skipping.')
                            filCount = 0
                            for root, dirs, files in os.walk(productName):
                                if files in files:
                                    filCount+=1
                            count = filCount
                    else:
                        dirCount=dirCount+1
                        print(productName,'Directory already exists! Do you want to overwrite?(y/n)')
                        answer = input()
                        # If saving directory overwrite options is set to yes.
                        if answer=='y' or answer=="yes":
                            for f in os.listdir(productName):
                                os.remove(os.path.join(productName, f))
                            for element in imageLinks:
                                filename=productName+'/'+str(count+1)+'.jpg'
                                r = requests.get(element, stream = True, headers=HEADERS)
                                if r.status_code == 200:
                                    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                                    r.raw.decode_content = True
                                    # Open a local file with wb ( write binary ) permission.
                                    with open(filename,'wb') as f:
                                        shutil.copyfileobj(r.raw, f)
                                    count=count+1
                            print(str(count)+' Images Downloaded Successfully')
                        # If saving directory overwrite options is set to no.
                        else:
                            filCount = 0
                            for root, dirs, files in os.walk(productName):
                                if files in files:
                                    filCount+=1
                            count = filCount
            products_info.loc[len(products_info.index)] = [productName,productCategory,productPrice,sellerName,returnpolicy,warranty,count]
        # If OutofStock/Currently Unavailable
        else:
            productName = soup.find('span',id='productTitle')
            productName = productName.text.replace('\n','')
            productCategory = soup.find('span',class_='nav-a-content')
            productCategory = remove_spaces(remove_backsklashn(productCategory.text))
            productPrice = 'Out of Stock'
            productNA = 'N.A.'
            products_info.loc[len(products_info.index)] = [productName,productCategory,productPrice,productNA,productNA,productNA,productNA]


Kindle (10th Gen), 6" Display with Built-in Light,WiFi (Black) Directory already exists! Do you want to overwrite?(y/n)
7 Images Downloaded Successfully
Kindle Paperwhite (10th gen) - with Built-in Light, Waterproof, 8 GB, WiFi Directory already exists! Do you want to overwrite?(y/n)
Kindle Paperwhite (10th gen) - with Built-in Light, Waterproof, 8 GB, WiFi Directory Overwriting.
7 Images Downloaded Successfully
Kindle (10th Gen), 6" Display with Built-in Light, WiFi (White) Directory already exists! Overwriting.
7 Images Downloaded Successfully
Kindle Paperwhite (10th gen) -with Built-in Light, Waterproof, 32 GB, WiFi + Free 4G LTE Directory already exists! Overwriting.
7 Images Downloaded Successfully
All-New Kindle Oasis (10th Gen) - Now with adjustable warm light, 7" Display, 8 GB, WiFi (Graphite) Directory already exists! Overwriting.
6 Images Downloaded Successfully
All-New Kindle Oasis (10th Gen) - Now with adjustable warm light, 7" Display, 32 GB, WiFi + Free 4G (Graphite) Dire

In [7]:
products_info

,name,category,price,seller,return_exchange,warranty,no_of_images
0,"Kindle (10th Gen), 6"" Display with Built-in Li...",Amazon Devices,7999.00,Cloud connect_SLP,10 days returnable,1 Year Warranty,7
1,Kindle Paperwhite (10th gen) - with Built-in L...,Amazon Devices,12999.00,Cloud connect_SLP,10 days returnable,1 Year Warranty,7
2,"Kindle (10th Gen), 6"" Display with Built-in Li...",Amazon Devices,7999.00,Cloud connect_SLP,10 days returnable,1 Year Warranty,7
3,Kindle Paperwhite (10th gen) -with Built-in Li...,Amazon Devices,17999.00,HARIOM COMMUNICATION LLP,10 days returnable,1 Year Warranty,7
4,All-New Kindle Oasis (10th Gen) - Now with adj...,Amazon Devices,21999.00,Cloud connect_SLP,10 days returnable,1 Year Warranty,6
5,All-New Kindle Oasis (10th Gen) - Now with adj...,Amazon Devices,28999.00,Cloud connect_SLP,10 days returnable,1 Year Warranty,6
6,All-New Kindle Oasis (10th Gen) - Now with adj...,Amazon Devices,24999.00,Cloud connect_SLP,10 days returnable,No-Contact Delivery,6
7,"Kindle Paperwhite 3G (7th gen), 6"" High Resolu...",Amazon Devices,13999.00,Cloud connect_SLP,Returns Policy,No-Contact Delivery,8
8,Kindle E-reader 8th generation!!,Books,Out of Stock,N.A.,N.A.,N.A.,N.A.
9,Kindle Reader,Books,Out of Stock,N.A.,N.A.,N.A.,N.A.


In [8]:
products_info.to_csv('products_info.csv')